In [3]:
import pandas as pd

books=pd.read_csv('books_categories.csv')

In [9]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    device=0,
    top_k=None
)


classifier("I love this!")


Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764602683484554},
  {'label': 'anger', 'score': 0.004419787786900997},
  {'label': 'sadness', 'score': 0.0020923942793160677},
  {'label': 'disgust', 'score': 0.0016119946958497167},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [10]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548407673835754},
  {'label': 'neutral', 'score': 0.16985219717025757},
  {'label': 'sadness', 'score': 0.11640916764736176},
  {'label': 'surprise', 'score': 0.020700708031654358},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.01516129169613123},
  {'label': 'anger', 'score': 0.0039351508021354675}]]

In [12]:
classifier(books["description"][0].split(","))

[[{'label': 'neutral', 'score': 0.7992554306983948},
  {'label': 'joy', 'score': 0.08890233188867569},
  {'label': 'surprise', 'score': 0.07869036495685577},
  {'label': 'fear', 'score': 0.011926419101655483},
  {'label': 'anger', 'score': 0.010722405277192593},
  {'label': 'sadness', 'score': 0.005871808622032404},
  {'label': 'disgust', 'score': 0.00463114632293582}],
 [{'label': 'surprise', 'score': 0.8200346231460571},
  {'label': 'fear', 'score': 0.07336509227752686},
  {'label': 'neutral', 'score': 0.05585900694131851},
  {'label': 'joy', 'score': 0.038931705057621},
  {'label': 'anger', 'score': 0.005518761463463306},
  {'label': 'disgust', 'score': 0.0033061767462641},
  {'label': 'sadness', 'score': 0.0029845992103219032}],
 [{'label': 'neutral', 'score': 0.7798488140106201},
  {'label': 'surprise', 'score': 0.0958196297287941},
  {'label': 'disgust', 'score': 0.03609604388475418},
  {'label': 'sadness', 'score': 0.03586896136403084},
  {'label': 'joy', 'score': 0.035305395722

In [24]:
import numpy as np

emotions_labels=["anger","disgust","fear","joy","sadness","surprise","nuetral"]
isbn=[]
emotion_score={label:[] for label in emotions_labels}

In [25]:
def calculate_max_emotion_scores(predictions):
    per_emotion_scores={label:[] for label in emotions_labels}
    for prediction in predictions:
        sorted_predictions=sorted(prediction,key=lambda x : x["label"])
        for index,label in enumerate(emotions_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label:float(np.max(scores)) for label,scores in per_emotion_scores.items()}

In [26]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences =books["description"][i].split(",")
    predictions =classifier(sentences)
    max_scores =calculate_max_emotion_scores(predictions)
    for label in emotion_score:
        emotion_score[label].append(float(max_scores[label]))

In [27]:
emotion_score

{'anger': [0.9816599488258362,
  0.2254915088415146,
  0.5101116895675659,
  0.4424884617328644,
  0.11093747615814209,
  0.2802099287509918,
  0.13731083273887634,
  0.004927455447614193,
  0.4454279839992523,
  0.021855611354112625],
 'disgust': [0.9818353652954102,
  0.39794233441352844,
  0.07449290156364441,
  0.2857910990715027,
  0.2691112160682678,
  0.8514364361763,
  0.027670297771692276,
  0.05470823496580124,
  0.2604818642139435,
  0.6185770630836487],
 'fear': [0.9615715742111206,
  0.7422993779182434,
  0.6967346668243408,
  0.19490236043930054,
  0.08158721029758453,
  0.029362503439188004,
  0.8089789748191833,
  0.1170528456568718,
  0.7239177823066711,
  0.20890235900878906],
 'joy': [0.8257947564125061,
  0.6778584122657776,
  0.795831024646759,
  0.7550956010818481,
  0.3736914098262787,
  0.031259942799806595,
  0.23376502096652985,
  0.011401821859180927,
  0.2767099142074585,
  0.18882213532924652],
 'sadness': [0.8734855651855469,
  0.5903540253639221,
  0.4471

In [29]:
from tqdm import tqdm

emotions_labels=["anger","disgust","fear","joy","sadness","surprise","nuetral"]
isbn=[]
emotion_score={label:[] for label in emotions_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences =books["description"][i].split(",")
    predictions =classifier(sentences)
    max_scores =calculate_max_emotion_scores(predictions)
    for label in emotion_score:
        emotion_score[label].append(float(max_scores[label]))

100%|██████████| 5197/5197 [06:09<00:00, 14.08it/s]


In [30]:
emotions_df=pd.DataFrame(emotion_score)
emotions_df["isbn13"]=isbn

In [31]:
emotions_df.head()

,anger,disgust,fear,joy,sadness,surprise,nuetral,isbn13
0,0.981660,0.981835,0.961572,0.825795,0.873486,0.965892,0.820035,9780002005883
1,0.225492,0.397942,0.742299,0.677858,0.590354,0.352532,0.158719,9780002261982
2,0.510112,0.074493,0.696735,0.795831,0.447140,0.074637,0.130888,9780006178736
3,0.442488,0.285791,0.194902,0.755096,0.712774,0.766087,0.146308,9780006280897
4,0.110937,0.269111,0.081587,0.373691,0.859045,0.498381,0.211765,9780006280934


In [32]:
books=pd.merge(books,emotions_df,on="isbn13")

In [33]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,simple_categories,predicted_categories_x,predicted_categories_y,anger,disgust,fear,joy,sadness,surprise,nuetral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Fiction,NaN,NaN,0.981660,0.981835,0.961572,0.825795,0.873486,0.965892,0.820035
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Fiction,Fiction,Fiction,0.225492,0.397942,0.742299,0.677858,0.590354,0.352532,0.158719
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Fiction,NaN,NaN,0.510112,0.074493,0.696735,0.795831,0.447140,0.074637,0.130888
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,Nonfiction,Nonfiction,Nonfiction,0.442488,0.285791,0.194902,0.755096,0.712774,0.766087,0.146308
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,Nonfiction,Nonfiction,Nonfiction,0.110937,0.269111,0.081587,0.373691,0.859045,0.498381,0.211765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Fiction,Fiction,Fiction,0.301649,0.137945,0.784776,0.790712,0.881092,0.959901,0.158956
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Nonfiction,Nonfiction,Nonfiction,0.039456,0.097942,0.025582,0.352703,0.799894,0.020313,0.187916
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,Fiction,Fiction,Fiction,0.326130,0.147783,0.918343,0.990661,0.797602,0.096878,0.555701
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,Nonfiction,NaN,NaN,0.053104,0.074542,0.130399,0.219269,0.946667,0.169134,0.074918


In [34]:
books.to_csv("books_with_emotions.csv",index=False)